# Prompt Chaining

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

#Basic Prompt Chaining

In [6]:
from langchain_core.prompts import PromptTemplate


story_prompt = PromptTemplate(
    input_variables=["genre"],
    template="Write a short {genre} story in 3-4 sentences."
)

summary_prompt = PromptTemplate(
    input_variables=["story"],
    template="Summarize the following story in one sentence:\n{story}"
)

In [7]:
def story_chain(genre):
    """Generate a story and its summary based on a given genre.

    Args:
        genre (str): The genre of the story to generate.

    Returns:
        tuple: A tuple containing the generated story and its summary.
    """
    story = (story_prompt | llm).invoke({"genre": genre}).content
    summary = (summary_prompt | llm).invoke({"story": story}).content
    return story, summary

In [8]:
genre = "science fiction"
story, summary = story_chain(genre)
print(f"Story: {story}\n\nSummary: {summary}")

Story: As the last star in the universe died, humanity's final hope, the generation ship Aurora, hurtled through the empty void, carrying the remnants of civilization to a distant planet. But as the ship's captain, Emma, gazed out at the blackness, she felt a strange energy building in the darkness, a power that threatened to consume them all. Suddenly, the ship lurched and shuddered, and Emma's worst fears were realized: they were not alone in the universe after all. A massive, ancient being stirred from the depths of space, its presence sending shockwaves through the ship and shattering the fragile hopes of humanity's survival.

Summary: As the last star in the universe dies, the generation ship Aurora, carrying humanity's remnants, is suddenly confronted with the threat of an ancient, massive being that has been awakened from the depths of space, shattering humanity's final hopes of survival.


#Sequential prompting

In [9]:
theme_prompt = PromptTemplate(
    input_variables=["text"],
    template="Identify the main theme of the following text:\n{text}"
)

tone_prompt = PromptTemplate(
    input_variables=["text"],
    template="Describe the overall tone of the following text:\n{text}"
)

takeaway_prompt = PromptTemplate(
    input_variables=["text", "theme", "tone"],
    template="Given the following text with the main theme '{theme}' and tone '{tone}', what are the key takeaways?\n{text}"
)

In [10]:
def analyze_text(text):
    """Perform a multi-step analysis of a given text.

    Args:
        text (str): The text to analyze.

    Returns:
        dict: A dictionary containing the theme, tone, and key takeaways of the text.
    """
    theme = (theme_prompt | llm).invoke({"text": text}).content
    tone = (tone_prompt | llm).invoke({"text": text}).content
    takeaways = (takeaway_prompt | llm).invoke({"text": text, "theme": theme, "tone": tone}).content
    return {"theme": theme, "tone": tone, "takeaways": takeaways}

In [11]:
sample_text = "The rapid advancement of artificial intelligence has sparked both excitement and concern among experts. While AI promises to revolutionize industries and improve our daily lives, it also raises ethical questions about privacy, job displacement, and the potential for misuse. As we stand on the brink of this technological revolution, it's crucial that we approach AI development with caution and foresight, ensuring that its benefits are maximized while its risks are minimized."

analysis = analyze_text(sample_text)
for key, value in analysis.items():
    print(f"{key.capitalize()}: {value}\n")

Theme: The main theme of the text is the need for responsible development and deployment of Artificial Intelligence (AI) to ensure that its benefits are maximized while its risks are minimized.

Tone: The overall tone of this text is cautionary and reflective. The author is acknowledging the potential benefits of artificial intelligence, but is also expressing concern about the risks and ethical implications associated with its development. The use of words like "concern", "caution", and "risks" convey a sense of serious consideration and prudence. The tone is also somewhat neutral, as the author is presenting a balanced view of the issue, acknowledging both the potential benefits and drawbacks of AI. Overall, the tone is thoughtful and deliberative, encouraging readers to approach the topic with a critical and nuanced perspective.

Takeaways: The key takeaways from the text are:

1. **Responsible development of AI is crucial**: The text emphasizes the need to approach AI development w

#Dynamic Prompt Generation

In [12]:
answer_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question concisely:\n{question}"
)

follow_up_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Based on the question '{question}' and the answer '{answer}', generate a relevant follow-up question."
)

In [13]:
def dynamic_qa(initial_question, num_follow_ups=3):
    """Conduct a dynamic Q&A session with follow-up questions.

    Args:
        initial_question (str): The initial question to start the Q&A session.
        num_follow_ups (int): The number of follow-up questions to generate.

    Returns:
        list: A list of dictionaries containing questions and answers.
    """
    qa_chain = []
    current_question = initial_question

    for _ in range(num_follow_ups + 1):  # +1 for the initial question
        answer = (answer_prompt | llm).invoke({"question": current_question}).content
        qa_chain.append({"question": current_question, "answer": answer})
        
        if _ < num_follow_ups:  # Generate follow-up for all but the last iteration
            current_question = (follow_up_prompt | llm).invoke({"question": current_question, "answer": answer}).content

    return qa_chain

In [14]:
initial_question = "What are the potential applications of quantum computing?"
qa_session = dynamic_qa(initial_question)

for i, qa in enumerate(qa_session):
    print(f"Q{i+1}: {qa['question']}")
    print(f"A{i+1}: {qa['answer']}\n")

Q1: What are the potential applications of quantum computing?
A1: The potential applications of quantum computing include:

1. **Cryptography**: Unbreakable encryption for secure data transmission.
2. **Optimization**: Faster solutions for complex optimization problems in fields like finance, logistics, and energy management.
3. **Simulation**: Accurate simulations of complex systems, such as molecules and materials, for chemistry and physics research.
4. **Machine Learning**: Improved algorithms for machine learning and artificial intelligence.
5. **Data Analysis**: Faster and more efficient data processing and analysis for fields like medicine, climate modeling, and genomics.
6. **Materials Science**: Design and simulation of new materials with unique properties.
7. **Chemical Synthesis**: Optimal design of chemical reactions and synthesis pathways.
8. **Financial Modeling**: Faster and more accurate modeling of complex financial systems.

These are just a few examples of the many po

#Error Handling and Validation

In [15]:
generate_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Generate a 4-digit number related to the topic: {topic}. Respond with ONLY the number, no additional text."
)

validate_prompt = PromptTemplate(
    input_variables=["number", "topic"],
    template="Is the number {number} truly related to the topic '{topic}'? Answer with 'Yes' or 'No' and explain why."
)

In [16]:
import re

def extract_number(text):
    """Extract a 4-digit number from the given text.

    Args:
        text (str): The text to extract the number from.

    Returns:
        str or None: The extracted 4-digit number, or None if no valid number is found.
    """
    match = re.search(r'\b\d{4}\b', text)
    return match.group() if match else None

In [17]:
def robust_number_generation(topic, max_attempts=3):
    """Generate a topic-related number with validation and error handling.

    Args:
        topic (str): The topic to generate a number for.
        max_attempts (int): Maximum number of generation attempts.

    Returns:
        str: A validated 4-digit number or an error message.
    """
    for attempt in range(max_attempts):
        try:
            response = (generate_prompt | llm).invoke({"topic": topic}).content
            number = extract_number(response)
            
            if not number:
                raise ValueError(f"Failed to extract a 4-digit number from the response: {response}")
            
            # Validate the relevance
            validation = (validate_prompt | llm).invoke({"number": number, "topic": topic}).content
            if validation.lower().startswith("yes"):
                return number
            else:
                print(f"Attempt {attempt + 1}: Number {number} was not validated. Reason: {validation}")
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
    
    return "Failed to generate a valid number after multiple attempts."

In [18]:
topic = "World War II"
result = robust_number_generation(topic)
print(f"Final result for topic '{topic}': {result}")

Final result for topic 'World War II': 1945
